In [ ]:
from tqdm import tqdm
import numpy as np
import pandas as pd
from lxml import html
import requests
import re
import csv
from sklearn.model_selection import train_test_split
from elasticsearch import Elasticsearch
from elasticsearch import helpers
indexing_distinct_tables = Elasticsearch(timeout=30, max_retries=10, retry_on_timeout=True)

In [ ]:
train_articles = pd.read_csv('data_articles_train.csv', delimiter=',')

In [ ]:
def search_index_random():
    
    result= indexing_distinct_tables.search(
        index="distinct_tables", 
        body = {
        "_source": ["tablePgID","tablePgTitle","tableSectionTitle","tableCaption","tableHeader","tableBody"],
        "from" : 0,
        "size" : 100,
        "query": {
            "function_score": {
                "random_score": {}
            }
        }
    })
    
    return result

In [ ]:
random_negatives = []

for i in tqdm(range(len(train_articles))):
    
    row = train_articles.iloc[i]
    articleID = row['page_id']
    articleTitle = row['page_title']
    
    #getting the table title match
    contNoMatch = 0
    result = search_index_random()
    max_score = result['hits']['max_score']
    
    for hit in result['hits']['hits']:
    
        tableID = hit['_source']['tablePgID']
        tableTitle = (hit['_source']['tablePgTitle'])
        #tableSectionTitle = (hit['_source']['tableSectionTitle'])
        #tableCaption = (hit['_source']['tableCaption'])
        #tableHeader = (hit['_source']['tableHeader'])
        #tableBody = (hit['_source']['tableBody'])
        
        #getting match data
        if (articleID != tableID): 
            
            if contNoMatch < 1:
            
                tableIDMatch = tableID
                tableTitleMatch = tableTitle
                #tableSectionTitleMatch = tableSectionTitle
                #tableCaptionMatch = tableCaption
                #tableHeaderMatch = tableHeader
                #tableBodyMatch = tableBody
    
                new_row = {"article_id": articleID,
                           "article_title": articleTitle,
                           "table_id": tableIDMatch,
                           "table_title": tableTitleMatch,
                           "label": '0'}
            
                random_negatives.append(new_row)
                
                contNoMatch = contNoMatch + 1
                
            else:
                
                contNoMatch = 0
                break

In [ ]:
df_random_negative = pd.DataFrame(random_negatives)
df_random_negative.to_csv('random_negative_1M_single_title.csv',index=False)